<a href="https://colab.research.google.com/github/EdoardoMorucci/Plant-Leaves-Search-Engine---MIRCV/blob/main/model_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
This notebook describes the fine-tuning process of Convolutional Neural Network using as Base Network DenseNet

# Connection to Google Drive

In [14]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


The dataset is on Google Drive and the dataset directory has the structure:

```
dataset/
  class_1/
    image_1.jpg
    image_2.jpg
    ...
  class_2/
    image_3.jpg
    image_4.jpg
    ...
  ...
  ...
  class_n/
    ...
```

# Import

In [ ]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt

# Data Preparation


In [18]:
! pip install -q kaggle

from google.colab import files
_ = files.upload()

! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [1]:
! kaggle datasets download -d davidedemarco/healthy-unhealthy-plants-dataset-segmented --unzip

100% 642M/642M [00:04<00:00, 73.5MB/s]



In [20]:
! mv /content/Healthy_and_Unhealthy_PlantsDataset_Segmented /content/gdrive/Shareddrives/MIRCV-PlantLeavesSearchEngine

^C


To train and test the model, we need three subsets: train, test and validation. To split the dataset, we use the [split-folder](https://pypi.org/project/split-folders/) package.

In [ ]:
!pip install split-folders tqdm

We need to check if the hardware accelaration is enabled, since training a CNN on a CPU could be infeasible.

In [ ]:
#check hardware acceleration
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


All'interno del file sysyem di GDrive ci posizioniamo nella cartella riservata al progetto e definiamo come costanti la cartella dove il dataset è presente e la cartella dove train test, validation test e test set saranno salvati.

In [3]:
BASE_DIR = "gdrive/Shareddrives/MIRCV-PlantLeavesSearchEngine/"
DATA_DIR = '/content/Healthy-and-Unhealthy-Plants-Dataset-Segmented'
SETS_DIR = '/content/healthy-unhealthy-plants-sets'
BATCH_SIZE = 256


When working with lots of real-world image data, corrupted images are a common occurence. Let's filter out badly-encoded images that do not feature the string "JFIF" in their header.

We need to create data splits. The dataset will be divided 80% in training set, 10% in validation set and 10% in test set.

In [4]:
import splitfolders
# split data
splitfolders.ratio(DATA_DIR, output=SETS_DIR, seed=123, ratio=(0.8, 0.1, 0.1), group_prefix=None)

Copying files: 72034 files [00:18, 3871.66 files/s]


Conto i file nelle cartelle

In [17]:
# simple version for working with CWD
print(os.listdir(DATA_DIR))
total = 0
for folder_name in os.listdir(DATA_DIR):
  len_dataset = len([name for name in os.listdir(DATA_DIR + '/' + folder_name)])
  total += len_dataset


['Apple', 'Blueberry', 'Cherry', 'Corn', 'Grape', 'Orange', 'Peach', 'Pepper', 'Potato', 'Raspberry', 'Soybean', 'Squash', 'Strawberry']
4159
1969
2489
5058
5521
7255
3634
3240
2868
635
4212
0
60
